In [3]:
# %load /home/hwixley/Documents/Year3/IAML/cwk2/INFR10069-2020-CW2/helpers/iaml01cw2_helpers.py
#
#  Helper functions for IAML 2020/21 cw2
#
import os
import gzip
import numpy as np

#
#  This function is after  https://github.com/zalandoresearch/fashion-mnist
#
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

#
#------ IAML original functions ---------------
#
def load_FashionMNIST(data_path):
    Xtrn, Ytrn = load_mnist(data_path, "train")
    Xtst, Ytst = load_mnist(data_path, "t10k")
    return Xtrn.astype(np.float), Ytrn, Xtst.astype(np.float), Ytst

def load_CoVoST2(data_path):
    data = np.load(data_path+'/speech_trn.npz', allow_pickle=True)
    Xtrn, Ytrn = data['Xtrn'], data['Ytrn']
    data = np.load(data_path+'/speech_tst.npz', allow_pickle=True)
    Xtst, Ytst = data['Xtst'], data['Ytst']
    return Xtrn, Ytrn, Xtst, Ytst


In [54]:

##########################################################
#  Python script template for Question 3 (IAML Level 10)
#  Note that:
#  - You should not change the filename of this file, 'iaml01cw2_q3.py', which is the file name you should use when you submit your code for this question.
#  - You should define the functions shown below in your code.
#  - You can define function arguments (parameters) and returns (attributes) if necessary.
#  - In case you define helper functions, do not define them here, but put them in a separate Python module file, "iaml01cw2_my_helpers.py", and import it in this script.
#  - For those questions requiring you to show results in tables, your code does not need to present them in tables - just showing them with print() is fine.
#  - You do not need to include this header in your submission
##########################################################

#--- Code for loading the data set and pre-processing --->
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

#Load the data:
import os
dataPath = os.path.join(os.getcwd(),'../data')
Xtrn, Ytrn, Xtst, Ytst = load_CoVoST2(dataPath)
#<----

# Q3.1
def iaml01cw2_q3_1():
    from sklearn.cluster import KMeans
    import math
    
    km = KMeans(n_clusters=22, random_state =1)
    km.fit(Xtrn)
    centers = km.cluster_centers_
    
    sumSquaredDists = 0
    
    for row in range(Xtrn.shape[0]):
        cluster = km.labels_[row]
        dist = math.sqrt(sum((centers[cluster] - Xtrn[row,:])**2))
        sumSquaredDists += dist**2
        
    print("Sum of squared distances (Euclidean) of samples to their closest cluster center:")
    print(round(sumSquaredDists,3))
    print()
    
    frequencies = np.zeros(22)
    for l in km.labels_:
        frequencies[l] += 1
        
    print("Number of samples for each cluster:")
    for i in range(22):
        print("Cluster " + str(i+1) + " = " + str(frequencies[i]))
        
#
# iaml01cw2_q3_1()   # comment this out when you run the function

# Q3.2
#def iaml01cw2_q3_2():
#
# iaml01cw2_q3_2()   # comment this out when you run the function

# Q3.3
#def iaml01cw2_q3_3():
#
# iaml01cw2_q3_3()   # comment this out when you run the function

# Q3.4
#def iaml01cw2_q3_4():
#
# iaml01cw2_q3_4()   # comment this out when you run the function

# Q3.5
#def iaml01cw2_q3_5():
#
# iaml01cw2_q3_5()   # comment this out when you run the function



In [55]:
iaml01cw2_q3_1()

Sum of squared distances (Euclidean) of samples to their closest cluster center:
38185.817

Number of samples for each cluster:
Cluster 1 = 1018.0
Cluster 2 = 1125.0
Cluster 3 = 1191.0
Cluster 4 = 890.0
Cluster 5 = 1162.0
Cluster 6 = 1332.0
Cluster 7 = 839.0
Cluster 8 = 623.0
Cluster 9 = 1400.0
Cluster 10 = 838.0
Cluster 11 = 659.0
Cluster 12 = 1276.0
Cluster 13 = 121.0
Cluster 14 = 152.0
Cluster 15 = 950.0
Cluster 16 = 1971.0
Cluster 17 = 1251.0
Cluster 18 = 845.0
Cluster 19 = 896.0
Cluster 20 = 930.0
Cluster 21 = 1065.0
Cluster 22 = 1466.0
